## Megatron-LM张量并行

Megatron是由 NVIDIA 的应用深度学习研究团队开发的大型、强大的Transformer模型，主要针对大规模训练大型 transformer 语言模型的研究。其主要贡献时提出了将模型进行横向分割而进行张量并行的思想。

关于Tensor parallelism的中文解读可以参考：

英伟达中国：https://zhuanlan.zhihu.com/p/420908718

知乎大佬对具体切分方法的图示+伪代码：https://zhuanlan.zhihu.com/p/366906920

博客园罗西的思考：https://www.cnblogs.com/rossiXYZ/p/15840803.html

这一节Notebook主要想介绍针对一个比较简单的模型，如何使用Megatron快速将其进行张量并行的部署

首先，安装megatron库：

In [ ]:
!git clone https://github.com/NVIDIA/Megatron-LM.git
!cd Megatron-LM
%pip install -v -e .

测试安装是否正确

In [3]:
import megatron

print(megatron)

<module 'megatron' from '/mnt/configblob/users/ruizhe/Megatron-LM/megatron/__init__.py'>


接下来的代码部分引用了Megatron单元测试的代码：https://github.com/NVIDIA/Megatron-LM/tree/main/tests/unit_tests

In [ ]:
import os
import torch
import megatron.core.parallel_state as ps
from megatron.core.tensor_parallel.data import broadcast_data

class Utils:

    world_size = torch.cuda.device_count()
    rank = int(os.environ['LOCAL_RANK'])

    @staticmethod
    def initialize_distributed():
        print(f'Initializing torch.distributed with rank: {Utils.rank}, world_size: {Utils.world_size}')
        torch.cuda.set_device(Utils.rank % torch.cuda.device_count())
        init_method = 'tcp://'
        master_ip = os.getenv('MASTER_ADDR', 'localhost')
        master_port = os.getenv('MASTER_PORT', '6000')
        init_method += master_ip + ':' + master_port
        torch.distributed.init_process_group(backend='nccl', world_size=Utils.world_size, rank=Utils.rank, init_method=init_method)
        
    @staticmethod
    def destroy_model_parallel():
        ps.destroy_model_parallel()
        torch.distributed.barrier()

    @staticmethod
    def initialize_model_parallel(tensor_model_parallel_size = 1, pipeline_model_parallel_size = 1, virtual_pipeline_model_parallel_size = None, pipeline_model_parallel_split_rank = None):
        ps.destroy_model_parallel()
        if not torch.distributed.is_initialized():
            Utils.initialize_distributed()
        ps.initialize_model_parallel(tensor_model_parallel_size, pipeline_model_parallel_size, virtual_pipeline_model_parallel_size, pipeline_model_parallel_split_rank)
        
def test_broadcast_data():
    Utils.initialize_model_parallel(2,4)
    input_data = {
        0 : torch.ones((8,8)).cuda() * 0.0,
        1 : torch.ones((8,8)).cuda() * 1.0,
        2 : torch.ones((8,8)).cuda() * 2.0,
        3 : torch.ones((8,8)).cuda() * 3.0,
        4 : torch.ones((8,8)).cuda() * 4.0,
        5 : torch.ones((8,8)).cuda() * 5.0,
        6 : torch.ones((8,8)).cuda() * 6.0,
        7 : torch.ones((8,8)).cuda() * 7.0
        }
    dtype = torch.float32
    actual_output = broadcast_data([0,1],input_data, dtype)
    assert(torch.equal(actual_output[0], input_data[0]))
    assert(torch.equal(actual_output[1], input_data[1]))
    Utils.destroy_model_parallel()

这部分仍需要使用多卡来启动程序

In [7]:
!torchrun --nproc_per_node=8 test_megatron_broadcast.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Initializing torch.distributed with rank: 1, world_size: 8
Initializing torch.distributed with rank: 3, world_size: 8
Initializing torch.distributed with rank: 7, world_size: 8
Initializing torch.distributed with rank: 4, world_size: 8
Initializing torch.distributed with rank: 2, world_size: 8
Initializing torch.distributed with rank: 5, world_size: 8
Initializing torch.distributed with rank: 6, world_size: 8
Initializing torch.distributed with rank: 0, world_size: 8
